In [ ]:
import pandas as pd
import numpy as np
import cv2
import requests
from io import BytesIO
from tqdm import tqdm

# ================================
# 1️⃣ Load and Prepare Data
# ================================
print("🔹 Loading report data...")
df = pd.read_csv("20250514-report.csv", parse_dates=['date_captured'])
df.columns = df.columns.str.strip().str.replace('"', '')

print(f"✅ Report loaded with {len(df)} rows.")

# ================================
# 2️⃣ Filter Labels and Off-Location Ends
# ================================
print("🔹 Filtering labels and Off-Location ends...")

labels_df = df[df['sku'] != '-'].copy()
off_ends_df = df[(df['sku'] == '-') & (df['end_location'].str.lower().str.contains('off', na=False))].copy()

print(f"✅ Found {len(labels_df)} labels and {len(off_ends_df)} Off-Location ends.")

# ================================
# 3️⃣ Load Images from URL (Reusable)
# ================================
def load_image_from_url(url, color=cv2.IMREAD_GRAYSCALE):
    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        img_array = np.asarray(bytearray(response.content), dtype=np.uint8)
        return cv2.imdecode(img_array, color)
    except:
        return None

# ================================
# 4️⃣ Visual-First Matching Function
# ================================
def visual_first_match(label_row, ends_df, max_template_ratio=0.5, min_visual_score=0.6):
    visit_id = label_row['visit_id']
    label_url = label_row['label_image_url']
    label_time = label_row['date_captured']
    label_brand = label_row['brand']

    candidates = ends_df[ends_df['visit_id'] == visit_id].copy()
    if candidates.empty:
        return np.nan, np.nan, np.nan, '❌ No Ends'

    label_img = load_image_from_url(label_url)
    if label_img is None:
        return np.nan, np.nan, np.nan, '❌ Label Image Missing'

    best_score = -1
    best_end_url = None
    best_time_diff = None

    for _, end_row in candidates.iterrows():
        end_img = load_image_from_url(end_row['end_image_url'])
        if end_img is None:
            continue

        h_ratio = label_img.shape[0] / end_img.shape[0]
        w_ratio = label_img.shape[1] / end_img.shape[1]
        if max(h_ratio, w_ratio) > max_template_ratio:
            scale = max_template_ratio / max(h_ratio, w_ratio)
            label_resized = cv2.resize(label_img, (int(label_img.shape[1]*scale), int(label_img.shape[0]*scale)))
        else:
            label_resized = label_img

        try:
            res = cv2.matchTemplate(end_img, label_resized, cv2.TM_CCOEFF_NORMED)
            _, max_val, _, _ = cv2.minMaxLoc(res)
        except:
            max_val = np.nan

        time_diff = abs((label_time - end_row['date_captured']).total_seconds()) / 60

        if max_val > best_score:
            best_score = max_val
            best_end_url = end_row['end_image_url']
            best_time_diff = time_diff

    if best_score >= min_visual_score:
        return best_end_url, best_time_diff, best_score, '✅ Visual Match'
    else:
        # Fallback to closest timestamp
        fallback = candidates.sort_values('date_captured').iloc[0]
        fallback_time_diff = abs((label_time - fallback['date_captured']).total_seconds()) / 60
        return fallback['end_image_url'], fallback_time_diff, np.nan, '⚠️ Timestamp Fallback'

# ================================
# 5️⃣ Run Visual-First Matching
# ================================
print("🔹 Running visual-first matching...")
tqdm.pandas(desc="Matching")
labels_df[['matched_end_image_url', 'timestamp_diff_min', 'opencv_template_score', 'match_type']] = labels_df.progress_apply(
    lambda row: pd.Series(visual_first_match(row, off_ends_df)), axis=1
)

print("✅ Visual-first matching complete.")

# ================================
# 6️⃣ Confidence Scoring
# ================================
print("🔹 Computing confidence scores...")

def brand_score(label_brand, product_name):
    if pd.isna(label_brand) or pd.isna(product_name):
        return 0.0
    return 1.0 if label_brand.lower() in product_name.lower() else 0.5

labels_df['brand_confidence'] = labels_df.apply(
    lambda row: brand_score(row['brand'], row['product_name']), axis=1
)

alpha, beta, gamma = 0.5, 0.3, 0.2
labels_df['final_confidence'] = (
    alpha * labels_df['opencv_template_score'].fillna(0) +
    beta * labels_df['timestamp_diff_min'].apply(lambda x: 1.0 if x <= 2 else 0.8 if x <=5 else 0.5).fillna(0) +
    gamma * labels_df['brand_confidence']
)

labels_df['confidence_flag'] = labels_df['final_confidence'].apply(
    lambda x: '✅ High' if x >= 0.85 else ('⚠️ Review' if x >= 0.6 else '❌ Low')
)

print("✅ Confidence scoring complete.")

# ================================
# 7️⃣ Save Final Output
# ================================
final_output_path = "off_location_visual_first_mapping.csv"
labels_df.to_csv(final_output_path, index=False)
print(f"✅ Final Off-Location visual-first mapping saved to: {final_output_path}")

# ================================
# 8️⃣ Sample Output
# ================================
print("\n🔍 Sample Results:")
print(labels_df[['label_image_url', 'matched_end_image_url', 'opencv_template_score', 'match_type', 'final_confidence', 'confidence_flag']].head())

/var/folders/l6/jvk_psqj1mz2tbsrq4j601_w0000gn/T/ipykernel_55475/3850503638.py:12: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv("20250514-report.csv", parse_dates=['date_captured'])


🔹 Loading report data...
✅ Report loaded with 4401 rows.
🔹 Filtering labels and Off-Location ends...
✅ Found 3313 labels and 506 Off-Location ends.
🔹 Running visual-first matching...


Matching:   0%|          | 0/3313 [00:00<?, ?it/s]

In [2]:
import pandas as pd
import numpy as np
import cv2
import requests
from io import BytesIO
from tqdm import tqdm

# ================================
# 1️⃣ Load Data
# ================================
print("🔹 Loading report data...")
df = pd.read_csv("20250514-report.csv", parse_dates=['date_captured'])
df.columns = df.columns.str.strip().str.replace('"', '')

# Filter for 50 labels and off-location ends
labels_df = df[df['sku'] != '-'].head(50).copy()
off_ends_df = df[(df['sku'] == '-') & (df['end_location'].str.lower().str.contains('off', na=False))].copy()

print(f"✅ 50 labels loaded | {len(off_ends_df)} off-location ends found.")

# ================================
# 2️⃣ Functions
# ================================
def load_image_from_url(url, color=cv2.IMREAD_GRAYSCALE):
    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        img_array = np.asarray(bytearray(response.content), dtype=np.uint8)
        return cv2.imdecode(img_array, color)
    except:
        return None

def visual_first_match(label_row, ends_df, max_template_ratio=0.5, min_visual_score=0.6):
    visit_id = label_row['visit_id']
    label_url = label_row['label_image_url']
    label_time = label_row['date_captured']

    candidates = ends_df[ends_df['visit_id'] == visit_id].copy()
    if candidates.empty:
        return np.nan, np.nan, np.nan, '❌ No Ends'

    label_img = load_image_from_url(label_url)
    if label_img is None:
        return np.nan, np.nan, np.nan, '❌ Label Image Missing'

    best_score, best_end_url, best_time_diff = -1, None, None
    for _, end_row in candidates.iterrows():
        end_img = load_image_from_url(end_row['end_image_url'])
        if end_img is None:
            continue

        h_ratio, w_ratio = label_img.shape[0] / end_img.shape[0], label_img.shape[1] / end_img.shape[1]
        if max(h_ratio, w_ratio) > max_template_ratio:
            scale = max_template_ratio / max(h_ratio, w_ratio)
            label_resized = cv2.resize(label_img, (int(label_img.shape[1]*scale), int(label_img.shape[0]*scale)))
        else:
            label_resized = label_img

        try:
            res = cv2.matchTemplate(end_img, label_resized, cv2.TM_CCOEFF_NORMED)
            _, max_val, _, _ = cv2.minMaxLoc(res)
        except:
            max_val = np.nan

        time_diff = abs((label_time - end_row['date_captured']).total_seconds()) / 60

        if max_val > best_score:
            best_score, best_end_url, best_time_diff = max_val, end_row['end_image_url'], time_diff

    if best_score >= min_visual_score:
        return best_end_url, best_time_diff, best_score, '✅ Visual Match'
    else:
        fallback = candidates.sort_values('date_captured').iloc[0]
        fallback_time_diff = abs((label_time - fallback['date_captured']).total_seconds()) / 60
        return fallback['end_image_url'], fallback_time_diff, np.nan, '⚠️ Timestamp Fallback'

# ================================
# 3️⃣ Run Matching
# ================================
print("🔹 Running visual-first matching on 50 labels...")
tqdm.pandas(desc="Matching")
labels_df[['matched_end_image_url', 'timestamp_diff_min', 'opencv_template_score', 'match_type']] = labels_df.progress_apply(
    lambda row: pd.Series(visual_first_match(row, off_ends_df)), axis=1
)

# ================================
# 4️⃣ Final Output
# ================================
labels_df.to_csv("off_location_visual_first_mapping_50.csv", index=False)
print("✅ Final mapping for 50 labels saved to: off_location_visual_first_mapping_50.csv")

print("\n🔍 Sample Results:")
print(labels_df[['label_image_url', 'matched_end_image_url', 'opencv_template_score', 'match_type']].head())

/var/folders/l6/jvk_psqj1mz2tbsrq4j601_w0000gn/T/ipykernel_55798/3708904178.py:12: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv("20250514-report.csv", parse_dates=['date_captured'])


🔹 Loading report data...
✅ 50 labels loaded | 506 off-location ends found.
🔹 Running visual-first matching on 50 labels...


Matching: 100%|██████████| 50/50 [00:00<00:00, 6145.50it/s]

✅ Final mapping for 50 labels saved to: off_location_visual_first_mapping_50.csv

🔍 Sample Results:
                                     label_image_url  matched_end_image_url  \
1  https://dtexg3-images.s3.ap-southeast-2.amazon...                    NaN   
2  https://dtexg3-images.s3.ap-southeast-2.amazon...                    NaN   
3  https://dtexg3-images.s3.ap-southeast-2.amazon...                    NaN   
4  https://dtexg3-images.s3.ap-southeast-2.amazon...                    NaN   
5  https://dtexg3-images.s3.ap-southeast-2.amazon...                    NaN   

   opencv_template_score match_type  
1                    NaN  ❌ No Ends  
2                    NaN  ❌ No Ends  
3                    NaN  ❌ No Ends  
4                    NaN  ❌ No Ends  
5                    NaN  ❌ No Ends  
